In [1]:
import re
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from secret import *
import chromadb

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def bert_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

raw_documents = WikipediaLoader(query="Atelectasis").load()
text_splitter = RecursiveCharacterTextSplitter(
          chunk_size = 200,
          chunk_overlap  = 20,
          length_function = bert_len,
          separators=['\n\n', '\n', ' ', ''],
      )
#print(raw_documents)
# documents = text_splitter.create_documents([raw_documents[0].page_content])

In [3]:
from langchain.indexes import GraphIndexCreator
from langchain.llms import OpenAI

In [4]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

In [5]:
graph = index_creator.from_text(raw_documents[0].page_content)

In [9]:
graph.get_topological_sort()

['Atelectasis',
 'Descartes',
 'Nevada',
 'a lung',
 'unilateral',
 'the alveoli are deflated down to little or no volume',
 'a collapsed lung',
 'a pneumothorax',
 'or as a result of surfactant deficiency',
 'infant respiratory distress syndrome',
 'normal exhalation',
 'various medical conditions',
 'post-op fever',
 'antique scooters',
 'mandolin',
 'state',
 'US',
 'gold']

In [43]:
# from IPython.display import SVG
# graph.draw_graphviz(filename="web.svg")

In [10]:
t = graph.get_triples()

In [14]:
t[0]

('Atelectasis', 'a lung', 'is the collapse or closure of')

In [45]:
from langchain.chains import GraphQAChain

In [46]:
chain = GraphQAChain.from_llm(OpenAI(temperature=0), graph=graph, verbose=True)

In [47]:
chain.run("What are the symptoms of Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis is the collapse or closure of a lung
Atelectasis is usually unilateral
Atelectasis is a condition where the alveoli are deflated down to little or no volume
Atelectasis is often called a collapsed lung
Atelectasis is not synonymous with a pneumothorax
Atelectasis may occur as a post-operative complication or as a result of surfactant deficiency
Atelectasis leads to infant respiratory distress syndrome
Atelectasis is caused by normal exhalation
Atelectasis is caused by various medical conditions
Atelectasis is associated with post-op fever

> Finished chain.


' Symptoms of Atelectasis include shortness of breath, chest pain, coughing, and difficulty breathing.'

In [37]:
documents = text_splitter.create_documents([raw_documents[0].page_content])

In [25]:
from langchain.indexes import GraphIndexCreator
from langchain.llms import OpenAI

In [38]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

In [39]:
graph = index_creator.from_text(documents[0].page_content)

In [28]:
graph

In [43]:
graph.draw_graphviz()

[('Atelectasis', 'a lung', 'is the collapse or closure of'),
 ('Atelectasis', 'reduced or absent gas exchange', 'results in'),
 ('Atelectasis', 'unilateral', 'is usually'),
 ('Atelectasis', 'part or all of one lung', 'affects'),
 ('Atelectasis', 'the alveoli are deflated down', 'is a condition where')]

In [13]:
from langchain.indexes import GraphIndexCreator
from langchain.llms import OpenAI

In [33]:
from langchain. graphs.neo4j_graph import Neo4jGraph

In [44]:
from IPython.display import SVG
graph.draw_graphviz(layout="dot", filename="web.svg")
SVG('web.svg')

ImportError: Could not import pygraphviz python package. Please install it with:`pip install pygraphviz`.

In [14]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

In [15]:
neo4j_vector = Neo4jVector.from_documents(
    documents,
    # hf,
    OpenAIEmbeddings(),
    url=NOE4J_URL,
    username="neo4j",
    password=NOE4J_PASSWORD,
)

In [16]:
query = "What is atelectasis?"
vector_results = neo4j_vector.similarity_search(query, k=2)
for i, res in enumerate(vector_results):
    print(res.page_content)
    if i != len(vector_results)-1:
        print()
vector_result = vector_results[0].page_content

Atelectasis is the collapse or closure of a lung resulting in reduced or absent gas exchange.  It is usually unilateral, affecting part or all of one lung. It is a condition where the alveoli are deflated down

Atelectasis is the collapse or closure of a lung resulting in reduced or absent gas exchange.  It is usually unilateral, affecting part or all of one lung. It is a condition where the alveoli are deflated down


In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

In [18]:
graph = Neo4jGraph(
        url = NOE4J_URL,
    username = 'neo4j',
    password = NOE4J_PASSWORD
)

print(graph.schema)


        Node properties are the following:
        [{'labels': 'Chunk', 'properties': [{'property': 'id', 'type': 'STRING'}, {'property': 'text', 'type': 'STRING'}, {'property': 'embedding', 'type': 'LIST'}]}]
        Relationship properties are the following:
        []
        The relationships are the following:
        []
        


In [19]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

graph_result = chain.run("What are the symptoms of Atelectasis")
graph_result



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Chunk)-[:HAS_SYMPTOM]->(s:Symptom)
WHERE c.text = 'Atelectasis'
RETURN s.property
Full Context:
[]

> Finished chain.


"I'm sorry, but I don't have the information to answer your question. Atelectasis is a medical condition characterized by the collapse or closure of a lung or a part of it, resulting in reduced or absent gas exchange. The symptoms of atelectasis can vary depending on the severity and location of the collapse. Common symptoms may include shortness of breath, rapid breathing, chest pain, coughing, and low oxygen levels. However, it is always best to consult a healthcare professional for an accurate diagnosis and appropriate treatment."